<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html exercisesweek41.do.txt  -->
<!-- dom:TITLE: Exercises week 41 -->

# Exercises week 41
**October 4-11, 2024**

Date: **Deadline is Friday October 11 at midnight**

# Overarching aims of the exercises this week

The aim of the exercises this week is to get started with implementing
gradient methods of relevance for project 2. This exercise will also
be continued next week with the addition of automatic differentation.
Everything you develop here will be used in project 2.

In order to get started, we will now replace in our standard ordinary
least squares (OLS) and Ridge regression codes (from project 1) the
matrix inversion algorithm with our own gradient descent (GD) and SGD
codes.  You can use the Franke function or the terrain data from
project 1. **However, we recommend using a simpler function like**
$f(x)=a_0+a_1x+a_2x^2$ or higher-order one-dimensional polynomials.
You can obviously test your final codes against for example the Franke
function. Automatic differentiation will be discussed next week.

You should include in your analysis of the GD and SGD codes the following elements
1. A plain gradient descent with a fixed learning rate (you will need to tune it) using the analytical expression of the gradients

2. Add momentum to the plain GD code and compare convergence with a fixed learning rate (you may need to tune the learning rate), again using the analytical expression of the gradients.

3. Repeat these steps for stochastic gradient descent with mini batches and a given number of epochs. Use a tunable learning rate as discussed in the lectures from week 39. Discuss the results as functions of the various parameters (size of batches, number of epochs etc)

4. Implement the Adagrad method in order to tune the learning rate. Do this with and without momentum for plain gradient descent and SGD.

5. Add RMSprop and Adam to your library of methods for tuning the learning rate.

The lecture notes from weeks 39 and 40 contain more information and code examples. Feel free to use these examples.

In summary, you should 
perform an analysis of the results for OLS and Ridge regression as
function of the chosen learning rates, the number of mini-batches and
epochs as well as algorithm for scaling the learning rate. You can
also compare your own results with those that can be obtained using
for example **Scikit-Learn**'s various SGD options.  Discuss your
results. For Ridge regression you need now to study the results as functions of  the hyper-parameter $\lambda$ and 
the learning rate $\eta$.  Discuss your results.

You will need your SGD code for the setup of the Neural Network and
Logistic Regression codes. You will find the Python [Seaborn
package](https://seaborn.pydata.org/generated/seaborn.heatmap.html)
useful when plotting the results as function of the learning rate
$\eta$ and the hyper-parameter $\lambda$ when you use Ridge
regression.

We recommend reading chapter 8 on optimization from the textbook of [Goodfellow, Bengio and Courville](https://www.deeplearningbook.org/). This chapter contains many useful insights and discussions on the optimization part of machine learning.

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [122]:
"""
# Settings #
"""
s = 999
np.random.seed(s)

Niterations = 1000
n = 100 # data points
fixed_eta = 0.03
Lambda  = 0.01

num_features = 3

"""
# Data #
"""
x = np.linspace(0,2,n)
x = x.reshape(n,1) # conform to shape code assumes
print(f"{x.shape=}")

noise = np.random.randn(n,1)
coefficients = np.random.randn(3,1)
a_0, a_1, a_2 = coefficients
y = a_0 + a_1*x + a_2 * x**2

# dataframe for nicer display of betas/thetas/coefficients/whatever
coeff_df = pd.DataFrame(coefficients, columns=["Real coefficients"], index=["a_0", "a_1", "a_2"])
display(coeff_df)

x.shape=(100, 1)


,Real coefficients
a_0,2.135532
a_1,-0.229976
a_2,0.073004


In [124]:
"""
# Calculations #
"""
# Design matrix including the intercept
# No scaling of data and all data used for training 
X = np.c_[np.ones((n,1)), x, x**2]

# Inverting manually
XT_X = X.T @ X
theta_linreg = np.linalg.inv(X.T @ X) @ (X.T @ y)
coeff_df["Manual inversion"] = theta_linreg

# Hessian matrix
H = (2.0/n)* XT_X
EigValues, EigVectors = np.linalg.eig(H)
print(f"Eigenvalues of Hessian Matrix: {EigValues}")

# problem-specific learning rate
eta = 1.0/np.max(EigValues)

display(coeff_df)

Eigenvalues of Hessian Matrix: [10.38415281  0.76122767  0.03183162]


,Real coefficients,Manual inversion
a_0,2.135532,2.135532
a_1,-0.229976,-0.229976
a_2,0.073004,0.073004


## Gradient solver class

In [13]:
class GradientRegression:
    def __init__(self, Lambda=0, learning_rate=0.05, max_iterations=1000, scaling=False):
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        # lambda=0 is equivalent to just using OLS, code runs but user should probably change it
        # not important if *not* intending to use any of the models that need it, so only warn later if that is the case
        self.Lambda = Lambda
        self.scaling = scaling

    def OLS_gradient(self, calculated_coefficients, X, y):
        gradient = (2.0/self.n_data)*X.T @ (X @ calculated_coefficients - y)
        return gradient
    
    def Ridge_gradient(self, calculated_coefficients, X, y):
        gradient = (2.0/self.n_data)*X.T @ (X @ calculated_coefficients - y) + 2*self.Lambda*calculated_coefficients
        return gradient
    
    def Lasso_gradient(self, calculated_coefficients, X, y):
        gradient = (2.0/self.n_data)*X.T @ (X @ calculated_coefficients - y) + self.Lambda*np.sign(calculated_coefficients)
        return gradient
    
    def GDfit(self, X, y, model_name = None, Lambda = None):
        n_data, num_features = X.shape
        # make accessible outside of class
        self.n_data = n_data
        self.num_features = num_features

        # ensure flag is set, it will be changed if the model is right
        using_lambda = False
        if Lambda is not None:
            self.Lambda = Lambda

        gradient = np.zeros(num_features)
        calculated_coefficients = np.random.randn(num_features,1)

        if model_name is None:
            raise ValueError("Must specify a model to fit with")
        elif model_name == "OLS":    
            selected_gradient_function = self.OLS_gradient
        elif model_name == "Ridge":
            selected_gradient_function = self.Ridge_gradient
            using_lambda = True
        elif model_name == "Lasso":
            selected_gradient_function = self.Lasso_gradient
            using_lambda = True
        else:
            raise ValueError("Invalid model name")
        
        if using_lambda and self.Lambda == 0:
            print("Warning: Ridge selected but hyperparameter Lambda is zero, result will be equivalent to OLS")
        
        # gradient loop
        for iter in range(self.max_iterations):
            # Gradient calculation - depends on selected model
            gradient = selected_gradient_function(calculated_coefficients, X, y)
            # Update calculated coefficients
            calculated_coefficients -= self.learning_rate*gradient

        # finished coefficient result
        self.calculated_coefficients = calculated_coefficients
        return calculated_coefficients

    def linear_predict(self, X):
        try:
            calculated_coefficients = self.calculated_coefficients
        except:
            raise ValueError("Coefficients have not been calculated, run a model fit first")
        
        y_predicted = X @ calculated_coefficients
        if self.scaling:
            y_tilde = y_tilde + self.y_mean

        return y_predicted
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))


## Basic Gradient Descent


In [169]:
lr_eta = 0.1
lr_lambda = 10**(-3)
GD_instance = GradientRegression(Lambda=lr_lambda, max_iterations=10000, learning_rate=eta)
for model in ["OLS", "Ridge", "Lasso"]:
    beta_linreg = GD_instance.GDfit(X, y, model_name = model)
    coeff_df[f"{model} parameters"] = beta_linreg

display(coeff_df)

,Real coefficients,Manual inversion,OLS parameters,Ridge parameters,Lasso parameters
a_0,2.135532,2.135532,2.135530,2.113702,2.119033
a_1,-0.229976,-0.229976,-0.229971,-0.183946,-0.187461
a_2,0.073004,0.073004,0.073002,0.053367,0.053217


unless lambda is $10^{-3}$ or smaller, ridge and lasso end up completely wrong regardless of learning rate

but then learning rate doesn't matter much

also max iterations 1000 instead of 10 000 does not converge and is wrong

OLS consistently gives the best results anyway

## Momentum based GD


## Stochastic Gradient Descent (SGD)

## Added Adagrad method

## Added RMSProp

## Added ADAM